### Modelling the problem

We have a set of employees, $E$, and a set of days, $D$.

Let $e \in E$ and $d \in D$.

$\forall e \in E$ and $\forall d \in D$, if employee $e$ has day $d$ off, $L (e,d) = 1$

Therefore $L$ is an $|E|$ by $|D|$ matrix.

$$
L_{e,d} = 
\begin{pmatrix}
a_{1,1} & a_{1,2} & \cdots & a_{1,|D|} \\
a_{2,1} & a_{2,2} & \cdots & a_{2,|D|} \\
\vdots & \vdots & \ddots & \vdots \\
a_{|E|,1} & a_{|E|,2} & \cdots & a_{|E|,|D|} 
\end{pmatrix}
$$

Let $|E| = 10$ and $|D| = 5$, meaning that there are 10 employees and 5 staff members.

We also have a set of employee quotas $Q$ for each day, which is a matrix of length $ |D| $. 

$Q (d)$ denotes the proportion of staff members who can take annual leave on day $ d $.

Therefore, 
$$\forall d \in D,  \sum_{e=0}^E L(e,d) \leq (|E| * Q(d))$$

Finally, there is a staff leave allowance $A$, which is a matrix of length $|E|$, where $A(e)$ denotes the holiday entitlement remaining for employee $e$.

Therefore, 
$$\forall e \in E, \sum_{d=0}^D L(e,d) \leq A(e)  $$

The preference of leave assignments for all employees in $E$ is given by an $E * D$ matrix $P$, where if $P(e,d) = 1$, the employee has requested this day off.

If an employee does not request a day off, then we do not want the algorithm to assign a day off to them.

Therefore,

$$ L(e,d) \leq P(e,d)  \quad \forall e \in E , \forall d \in D $$



In [689]:
from ortools.sat.python import cp_model
import numpy as np
import pandas as pd
from math import ceil
import random

In [690]:
# num_staff = 10
# num_days = 5

# # staff_limit = [((random.randrange(25, 50, 5) / 100)) for i in range(num_days)] # random capacity for each day

# staff_limit = [0.3 for i in range(num_days)] # random capacity for each day
# print(f"Staff Limit: {staff_limit}")

# # leave_allowance = [random.randint(0, 5) for i in range(num_staff)] # random leave allowance for each staff

# leave_allowance = [5 for i in range(num_staff)] # random leave allowance for each staff
# print(f"Staff Leave Allowance (number of days): {leave_allowance}")

# num_ones = int(num_staff * num_days * 0.5)
# num_zeros = (num_staff * num_days) - num_ones
# array = np.array([1] * num_ones + [0] * num_zeros)
# np.random.shuffle(array)
# p = array.reshape(num_staff,num_days)
# print(f"Preference Matrix: \n {p}")

# grades
# 1 = associate 0.3476 4
# 2 = senior associate 0.4251 4
# 3 = manager 0.1136 1
# 4 = senior manager 0.072 1
# 5 = director 0.0414
num_staff = 10
num_days = 10
daily_quotas = [2] * 10
grades = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  # staff grades

grade_daily_quotas = [
    [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],  #associate
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1], #senior associate
    [5, 5, 5, 5, 5, 5, 5, 5, 5, 5], #manager
    [5, 5, 5, 5, 5, 5, 5, 5, 5, 5], #senior manager
    [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]  #director
]
    

leave_allowance = [10] * 10
preference_matrix = [
    [1, 1, 1, 0, 0, 0, 0, 0, 0, 1],
    [0, 1, 1, 1, 0, 0, 0, 0, 0, 1],
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 1],
    [0, 0, 0, 1, 1, 1, 0, 0, 0, 1],
    [0, 0, 0, 0, 1, 1, 1, 0, 0, 1],
    [0, 0, 0, 0, 0, 1, 1, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
]


p = preference_matrix

print(f"Preference Matrix: \n {p}")



Preference Matrix: 
 [[1, 1, 1, 0, 0, 0, 0, 0, 0, 1], [0, 1, 1, 1, 0, 0, 0, 0, 0, 1], [0, 0, 1, 1, 1, 0, 0, 0, 0, 1], [0, 0, 0, 1, 1, 1, 0, 0, 0, 1], [0, 0, 0, 0, 1, 1, 1, 0, 0, 1], [0, 0, 0, 0, 0, 1, 1, 1, 0, 1], [0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]


In [691]:
model = cp_model.CpModel()

If employee $e$ has day $d$ off, $L (e,d) = 1$

In [692]:
l = {}
for e in range(num_staff):
    for d in range(num_days):
        l[(e, d)] = model.new_bool_var(f"L_{e}_d{d}")

Ensuring holiday entitlement is not exceeded (denoted by A for each employee).

$\forall e \in E, \sum_{d=0}^D L(e,d) \leq A(e)  $

In [693]:
for e in range(num_staff):
    model.Add(sum(l[e, d] for d in range(num_days)) <= leave_allowance[e])

Ensuring that maximum daily leave quota is not exceeded (denoted by array Q)

$\forall d \in D,  \sum_{e=0}^E L(e,d) \leq (|E| * Q(d))$

In [694]:

for d in range(num_days):

    for grade in range(5):

        staff_in_grade = [e for e in range(num_staff) if grades[e] == grade]

        grade_allocation = sum(l[e, d] for e in staff_in_grade)

        model.Add(grade_allocation <= grade_daily_quotas[grade][d])

    # model.Add(sum(l[e, d] for e in range(num_staff)) <= grade_daily_quotas[grades[e]][d])

Don't allocate leave that isn't requested

In [695]:
for d in range(num_days):
    for e in range(num_staff):
        if p[e][d] == 0:
            model.Add(l[e, d] == 0)

Max 2 consecutive weeks

In [696]:
# allow max 10 days in a row for each staff
for e in range(num_staff):
    for d in range(num_days - 10):
        model.Add(sum(l[e, d + i] for i in range(11)) <= 10)

### Multi-Objective solving

Two methods:
- add a weighting for each 
- solve one, then use it as a coefficient and solve again (lexographic solving)

Attempting with method 1

Objective functions:

1) Maximise the leave which matches a persons preference, and minimise leave that doesn't match a preference.

    The line below aims to maximise $P( e , d ) \times L( e , d )$ for each staff member 
    $ e \in E $ and for each day $ d \in d $ .

    If a staff member hasn't asked for a day off ( meaning $ P(e,d) = 0 $ ), the algorithm shouldn't give them a day off, as $ P( e , d ) \times L( e , d ) = 0 \times 1 = 0 $

    Likewise, if a staff member has asked for a day off ( meaning $ P(e,d) = 1 $ ), the algorithm should try to give them a day off, as $ P( e , d ) \times L( e , d ) = 1 \times 1 = 1 $




In [697]:
objective_1 = sum(p[e][d] * l[e, d] for e in range(num_staff) for d in range(num_days))

objective_1_weighting = 1


2) Maximise number of consecutive days and add an upper constraint to the number of consecutive days you can take (soft constraint)

    In order to encourage the algorithm to want to approve "blocks" of annual leave rather than individual days, we aim to maximise the number of consecutive days allocated.

    We can define a variable $S$, where $S(e,d) = L(e,d) * L(e,d+1)$ for $\sum_{d=0}^{|D|-1}$.

    For example, if a person has a preference of two consecutive days $\{1,1\}$, we aim to maximise the product $L(e,d)$ of these two consecutive days.

    Scheduling $\{1,0\}$ or $\{0,1\}$ results in a product of $0$, meaning that this behavour will be discouraged.

In [698]:
consecutive_days_off = {}

for e in range(num_staff):
    for d in range(num_days-1):
        consecutive_days_off[(e, d)] = model.new_bool_var(f"consecutive_days_off_{e}_{d}")

for e in range(num_staff):
    for d in range(num_days-1):
        model.add_multiplication_equality(consecutive_days_off[(e, d)], [l[e, d], l[e, d+1]])

objective_2 = sum(consecutive_days_off[(e, d)] for e in range(num_staff) for d in range(num_days-1))

objective_2_weighting = 5

3) Prioritise those who have more leave allowance left

In [699]:
objective_3 = sum(leave_allowance[e] * l[e, d] for e in range(num_staff) for d in range(num_days))
objective_3_weighting = 1

Join all objectives together with weightings and solve

In [700]:
model.maximize(
                (objective_1_weighting * objective_1)
                + (objective_2_weighting * objective_2) 
                 + (objective_3_weighting * objective_3)
            )   

In [701]:
# Solve model

# solver = cp_model.CpSolver()
# solver.parameters.enumerate_all_solutions = True
# solver.parameters.max_time_in_seconds = 30.0
# status = solver.Solve(model)

In [702]:

def highlight_cells(val):
    color = '#00ff15' if val == 1 else ''
    return 'color: %s' % color

# function taken from stack overflow
def highlight_diff(data, other, color='#ff616b'):
    attr = f'background-color: {color}'
    return pd.DataFrame(np.where(data.ne(other), attr, ''),
                        index=data.index, columns=data.columns)

# if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:

#     solutionArray = [[solver.Value(l[s,d]) for d in range(num_days)] for s in range(num_staff)]

#     df = pd.DataFrame(solutionArray, columns=[f'Day {d+1}' for d in range(num_days)], index=[f'Employee {x+1}' for x in range(num_staff)])

#     df2 = pd.DataFrame(p, columns=[f'Day {d+1}' for d in range(num_days)], index=[f'Employee {x+1}' for x in range(num_staff)])


#     print("Solution\nRed cells are cells where preference matrix and solution matrix differ.")
#     df_styled = df.style.apply(highlight_diff, axis=None, other=df2).applymap(highlight_cells)
#     display(df_styled)

#     print("Preference Matrix")
#     df2_styled = df2.style.applymap(highlight_cells)
#     display(df2_styled)

# else:
#     print("No feasible solution found.")

In [703]:

class SolutionPrinter(cp_model.CpSolverSolutionCallback):

    def __init__(self, num_staff, num_days, l, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._num_staff = num_staff
        self._num_days = num_days
        self._leave = l
        self._solution_count = 0
        self._solution_limit = limit


    def on_solution_callback(self):

        self._solution_count += 1

        print(f"Solution {self._solution_count}")

        solutionArray = [[self.value(self._leave[(s, d)]) for d in range(self._num_days)] for s in range(self._num_staff)]

        df = pd.DataFrame(solutionArray, columns=[f'Day {d+1}' for d in range(self._num_days)], index=[f'Employee {x+1}' for x in range(self._num_staff)])

        df2 = pd.DataFrame(p, columns=[f'Day {d+1}' for d in range(self._num_days)], index=[f'Employee {x+1}' for x in range(self._num_staff)])
        
        df_styled = df.style.applymap(highlight_cells).apply(highlight_diff, axis=None, other=df2)

        display(df_styled)
        print("Objective value: ", self.ObjectiveValue())

        preference_leave_days_count = df2.sum().sum()
        assigned_leave_days_count = df.sum().sum()
        print(f"Preference Leave Days Count: {preference_leave_days_count}")
        print(f"Assigned Leave Days Count: {assigned_leave_days_count}")

        print(f"Percentage of leave granted: {(assigned_leave_days_count / preference_leave_days_count) * 100:.2f}%")



        if self._solution_count >= self._solution_limit:
            print(f"Stop search after {self._solution_limit} solutions")
            self.stop_search()



    def solutionCount(self):
        return self._solution_count



In [704]:
solution_limit = 100
solution_printer = SolutionPrinter(num_staff, num_days, l, solution_limit)

solver = cp_model.CpSolver()
# solver.parameters.linearization_level = 0
solver.parameters.max_time_in_seconds = 60

solver.parameters.enumerate_all_solutions = True

solver.Solve(model, solution_printer)

# Statistics.
print("\nStatistics")
print(f"  - conflicts: {solver.num_conflicts}")
print(f"  - branches : {solver.num_branches}")
print(f"  - wall time: {solver.wall_time}s")

Solution 1


C:\Users\seren\AppData\Local\Temp\ipykernel_48144\337526799.py:24: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  df_styled = df.style.applymap(highlight_cells).apply(highlight_diff, axis=None, other=df2)


,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,Day 10
Employee 1,1,1,0,0,0,0,0,0,0,0
Employee 2,0,1,1,0,0,0,0,0,0,0
Employee 3,0,0,1,1,0,0,0,0,0,0
Employee 4,0,0,0,1,1,0,0,0,0,0
Employee 5,0,0,0,0,1,1,0,0,0,0
Employee 6,0,0,0,0,0,1,1,0,0,0
Employee 7,0,0,0,0,0,0,1,1,0,0
Employee 8,0,0,0,0,0,0,0,1,1,1
Employee 9,0,0,0,0,0,0,0,0,1,1
Employee 10,0,0,0,0,0,0,0,0,0,0


Objective value:  259.0
Preference Leave Days Count: 34
Assigned Leave Days Count: 19
Percentage of leave granted: 55.88%
Solution 2


,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,Day 10
Employee 1,1,1,1,0,0,0,0,0,0,0
Employee 2,0,1,1,1,0,0,0,0,0,0
Employee 3,0,0,0,0,0,0,0,0,0,0
Employee 4,0,0,0,1,1,0,0,0,0,0
Employee 5,0,0,0,0,1,1,0,0,0,0
Employee 6,0,0,0,0,0,1,1,0,0,0
Employee 7,0,0,0,0,0,0,1,1,0,0
Employee 8,0,0,0,0,0,0,0,1,1,1
Employee 9,0,0,0,0,0,0,0,0,1,1
Employee 10,0,0,0,0,0,0,0,0,0,0


Objective value:  264.0
Preference Leave Days Count: 34
Assigned Leave Days Count: 19
Percentage of leave granted: 55.88%
Solution 3


,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,Day 10
Employee 1,1,1,1,0,0,0,0,0,0,0
Employee 2,0,1,1,1,0,0,0,0,0,0
Employee 3,0,0,0,1,1,0,0,0,0,0
Employee 4,0,0,0,0,1,1,0,0,0,0
Employee 5,0,0,0,0,0,1,1,0,0,0
Employee 6,0,0,0,0,0,0,0,0,0,0
Employee 7,0,0,0,0,0,0,1,1,1,1
Employee 8,0,0,0,0,0,0,0,1,1,1
Employee 9,0,0,0,0,0,0,0,0,0,0
Employee 10,0,0,0,0,0,0,0,0,0,0


Objective value:  269.0
Preference Leave Days Count: 34
Assigned Leave Days Count: 19
Percentage of leave granted: 55.88%
Solution 4


,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,Day 10
Employee 1,1,1,1,0,0,0,0,0,0,0
Employee 2,0,1,1,1,0,0,0,0,0,0
Employee 3,0,0,0,0,0,0,0,0,0,0
Employee 4,0,0,0,1,1,1,0,0,0,0
Employee 5,0,0,0,0,1,1,1,0,0,0
Employee 6,0,0,0,0,0,0,0,0,0,0
Employee 7,0,0,0,0,0,0,1,1,1,1
Employee 8,0,0,0,0,0,0,0,1,1,1
Employee 9,0,0,0,0,0,0,0,0,0,0
Employee 10,0,0,0,0,0,0,0,0,0,0


Objective value:  274.0
Preference Leave Days Count: 34
Assigned Leave Days Count: 19
Percentage of leave granted: 55.88%

Statistics
  - conflicts: 1498
  - branches : 7829
  - wall time: 0.07338460000000001s
